In [1]:
#importing dependencies related to nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#importing dependencies related to image transformations
import torchvision
from torchvision import transforms
from PIL import Image

#importing dependencies related to data loading
from torchvision import datasets
from torch.utils.data import DataLoader

#importing Tensorboard for data visualization
from torch.utils.tensorboard import SummaryWriter

In [2]:
#defining image transformations

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

In [3]:
# loading data
data = {
    'train':
    datasets.ImageFolder(root='./data/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='./data/test', transform=image_transforms['test']),
}

# Dataloader iterators, used for making batches
dataloaders = {
    'train': DataLoader(data['train'], batch_size=100, shuffle=True),
    'test': DataLoader(data['test'], batch_size=100, shuffle=True)
}

In [4]:
#loading MobileNetv2
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)

Using cache found in C:\Users\Rakshit/.cache\torch\hub\pytorch_vision_v0.6.0


In [5]:
#freezing the initial layers of MobileNetv2
for param in model.parameters():
    param.requires_grad = False

In [6]:
#adding our own classifier
model.classifier[1] = nn.Sequential(
                      nn.Linear(1280, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 2),                   
                      nn.LogSoftmax(dim=1))

In [7]:
#checking trainable parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

328450


In [8]:
#checking GPU Avaibility
if torch.cuda.is_available():
    print('training on GPU')
    device = torch.device("cuda:0")
else:
    print('training on CPU')
    device = torch.device("cpu")

training on GPU


In [15]:
#training data
from tqdm.notebook import tqdm #for loading bars

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs,device):
    writer = SummaryWriter("saves/logs")
    for epoch in tqdm(range(epochs)):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in (val_loader):
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)          
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,valid_loss, num_correct / num_examples))
        
        #save model
        if(training_loss<0.005):
            torch.save(model,"saves/"+str(epoch)+".pth")
        
        #tensorBoard save log
        writer.add_scalar('Loss', loss.item(), epoch)
        writer.add_scalar('Train/Loss', training_loss, epoch)
        writer.add_scalar('Test/Loss', valid_loss, epoch)
        writer.add_scalar('Test/Accuracy',num_correct / num_examples, epoch)
        writer.flush()


In [11]:
#testing data
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloaders['test']:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [12]:
#put model to GPU (if available)
model=model.to(device)

In [13]:
#choosing optimizer and loss Function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [16]:
#train the model first 100 epochs
train(model, optimizer,loss,dataloaders['train'],dataloaders['test'],100,device)

Epoch: 0, Training Loss: 0.4110, Validation Loss: 0.1502, accuracy = 0.9970
Epoch: 1, Training Loss: 0.1368, Validation Loss: 0.0555, accuracy = 0.9970
Epoch: 2, Training Loss: 0.0925, Validation Loss: 0.0408, accuracy = 1.0000
Epoch: 3, Training Loss: 0.0728, Validation Loss: 0.0299, accuracy = 1.0000
Epoch: 4, Training Loss: 0.0638, Validation Loss: 0.0203, accuracy = 1.0000
Epoch: 5, Training Loss: 0.0527, Validation Loss: 0.0153, accuracy = 1.0000
Epoch: 6, Training Loss: 0.0498, Validation Loss: 0.0129, accuracy = 1.0000
Epoch: 7, Training Loss: 0.0478, Validation Loss: 0.0117, accuracy = 1.0000
Epoch: 8, Training Loss: 0.0416, Validation Loss: 0.0102, accuracy = 1.0000
Epoch: 9, Training Loss: 0.0465, Validation Loss: 0.0104, accuracy = 1.0000
Epoch: 10, Training Loss: 0.0384, Validation Loss: 0.0092, accuracy = 1.0000
Epoch: 11, Training Loss: 0.0387, Validation Loss: 0.0083, accuracy = 1.0000
Epoch: 12, Training Loss: 0.0464, Validation Loss: 0.0078, accuracy = 1.0000
Epoch: 13

In [17]:
#test the model
test_model(model)

correct: 330  total: 330
accuracy = 1.000000


In [18]:
#save the model
torch.save(model, "./firstHundread.pth")